In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [2]:
chunks = pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)

In [6]:
temp = next(chunks)

In [7]:
temp

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
10000,1508890831,1730477811,85,88,34.136,56.143
10001,1497933162,1730477837,14,15,88.933,85.091
10002,1508038365,1730477894,27,27,0.000,0.000
10003,1497929343,1730478017,13,13,135.231,98.537
10004,1508135005,1730478173,15,22,32.500,38.334
...,...,...,...,...,...,...
19995,1508864786,1740277326,24,24,50.417,71.030
19996,1497953673,1740277326,14,15,64.667,70.494
19997,1508170507,1740277326,11,11,49.636,48.778
19998,1508121518,1740277326,11,13,55.538,58.992


Filter from_npi to be entity type 1 and to_npi to be entity type 2
- will need to happen when we import the tables into the database and join the hop_table to the npi table.

In [10]:
# Filter the hop teaming data so that the transaction_count is at least 50 and the average_day_wait is less than 50
temp[(temp['transaction_count'] >= 50) & (temp['average_day_wait'] < 50)]

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
10000,1508890831,1730477811,85,88,34.136,56.143
10007,1508217811,1730478330,61,64,44.063,48.131
10018,1497970735,1730478827,36,113,16.867,25.390
10020,1508151945,1730478835,46,437,0.000,0.000
10023,1508087289,1730479122,52,53,28.226,66.420
...,...,...,...,...,...,...
19974,1508006412,1740277128,105,105,0.019,0.195
19975,1508866526,1740277128,137,138,0.000,0.000
19977,1508817529,1740277128,507,510,30.147,23.008
19986,1508133711,1740277201,41,67,43.567,44.373


In [ ]:
# Filter 

In [4]:
chunks2 = pd.read_csv('data/npidata_pfile_20050523-20220213.csv', chunksize = 10000)

In [5]:
next(chunks2)

C:\Users\hjrko\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17,34,72,76,80,84,87,89,90,91,93,94,95,97,98,99,101,102,103,105,106,163,171,175,178,179,181,182,183,185,186,189,190,191,193,194,197,198,201,205,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,...,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15,Certification Date
0,1679576722,1.0,NaN,NaN,NaN,WIEBE,DAVID,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588667638,1.0,NaN,NaN,NaN,PILCHER,WILLIAM,C,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1497758544,2.0,NaN,<UNAVAIL>,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1306849450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1215930367,1.0,NaN,NaN,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1700889045,1.0,NaN,NaN,NaN,PARKER,KATRINA,NaN,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,1619970951,1.0,NaN,NaN,NaN,GENOFF,MICHAEL,CHRISTOPHER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,1528061868,2.0,NaN,<UNAVAIL>,GREEN HILL INC,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,1437152774,1.0,NaN,NaN,NaN,MASON,ATHENA,C,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
